In [54]:
import os
import pandas as pd
import urllib.request
import requests
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Libraries imported.


Next I will import Folium to create maps and Beautifulsoup to parse through the website and get the table contents from the url.

In [2]:
! conda install -c anaconda beautifulsoup4 --yes
from bs4 import BeautifulSoup
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |           py37_0         162 KB  anaconda
    ca-certificates-2020.7.22  |                0         132 KB  anaconda
    certifi-2020.6.20          |           py37_0         159 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    soupsieve-2.0.1            |             py_0          33 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.3 MB

The following NEW packages will be I

Next I will import the raw geospatial coordinates since the geocoder was having issues

In [4]:
!ls ./Geospatial_Coordinates.csv
path = './Geospatial_Coordinates.csv'
df=pd.read_csv(path)
df.head(11)

./Geospatial_Coordinates.csv


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


This code is using Beautiful Soup(BS4) to parse through html tags to get the table content and store the data in a pandas dataframe

In [5]:
#The url we will use to get the postal codes
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#This is the request to get the info and then reading to get all the html info
html = urllib.request.urlopen(url).read()

#Soups cleans the html tags to make it easier to parse thru it
soup = BeautifulSoup(html,'html.parser')

#We use BS4 to find the table with the postal code. I was able to find the specific class to the table
result =  soup.find("table",{"class":'wikitable sortable'})

counter= 0 #Keeps track of position in table row
neighborhood = ''#Where we will store all neighborhoods in each postal code

#Finds all tables rows
rows = result.findAll('tr')
#Creates data to store the data
df2 = pd.DataFrame({"Postal Code":[], 
                    "Borough":[],
                    "Neighborhood":[] }) 


#Getting into table each row
for row in rows:
    #Gets all table data in the row
    cols = row.findAll('td')
    #gets into each data in the row and if it finds a postal code not assigned,breaks out of that specific loop
    #Other store the table data into a pandas dataframe
    for col in cols:
        if(col.findNext('td').text.strip() == 'Not assigned'):
            #print('This actually workssssssss')
            break
        if(counter == 0):
            pc = col.text.strip()
        if(counter ==1):
            bor= col.text.strip()
        if(counter >=2):
            neighborhood += col.text.strip()
        counter+=1
    if(counter>=2):
        
        tempDF = pd.DataFrame({"Postal Code":pc, 
                    "Borough":bor,
                    "Neighborhood": [neighborhood]}) 

        df2=df2.append(tempDF)
        neighborhood=''
    counter = 0
df2.head(60)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
0,M4A,North York,Victoria Village
0,M5A,Downtown Toronto,"Regent Park, Harbourfront"
0,M6A,North York,"Lawrence Manor, Lawrence Heights"
0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
0,M1B,Scarborough,"Malvern, Rouge"
0,M3B,North York,Don Mills
0,M4B,East York,"Parkview Hill, Woodbine Gardens"
0,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
#Store a copy of df2
dftest=df2
#Fixes the index values
dftest.reset_index(drop=True, inplace=True)
dftest.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Adds empty columns of lat and long into dataframe

In [13]:
a = dftest.iloc[0:,0]
t = 0
# a
#Adds two empty columns of Latitude and Longitude into a list
lis=pd.DataFrame({"Latitude":[],
                   "Longitude":[] })
#Another copy of the dataframe
dfTEST = dftest
#Adds empty columns of lat and long into dataframe without messing up order
dfTEST=dfTEST.append(lis,sort=False)

dfTEST.head(20)

#Some test code for changing pandas values
# for i in a:
#     #print(dfTEST.iloc[t,'Postal Code'],'test')
#     dfTEST.iloc[t,3]='new1 Works?'
#     dfTEST.iloc[t,4]='New2 Works?'
#     t+=1
#     print(i,t)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",NaN,NaN
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",NaN,NaN
7,M3B,North York,Don Mills,NaN,NaN
8,M4B,East York,"Parkview Hill, Woodbine Gardens",NaN,NaN
9,M5B,Downtown Toronto,"Garden District, Ryerson",NaN,NaN


In [14]:
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Sets values of Latitude and Longitude in dataframe based on Postal Code

In [15]:
dfPostal = df.iloc[0:,0]
dfPostal.head()
postalIndex = 0
df2Index = 0
for i in dfPostal:
    for p in a:
        if(p==i):
           # print('Found PostalCde Match',p,i)
            dfTEST.iloc[df2Index,3]= df.iloc[postalIndex,1]
            dfTEST.iloc[df2Index,4]= df.iloc[postalIndex,2]
            #print(postalIndex)
            df2Index=0
            postalIndex +=1
            break
        df2Index+=1
dfTEST.head()            

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [16]:
#           TEST CODE
#-------- Dont Not Read----------------
# ui = pd.Index(list(df))
# ui.get_loc('M1B')
# unique_index = pd.Index(list(dfPostal))
# print(unique_index)
# unique_index.get_loc('M1B')

In [17]:
dfTEST.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Plotting the neighborhood on Folium

In [18]:
# create map of Toronto using latitude and longitude values
torLat = 43.6532
torLon = -79.3832
map_toronto = folium.Map(location=[torLat, torLon], zoom_start=11)

# add markers to map of Toronto city
for lat, lng, label in zip(dfTEST['Latitude'], dfTEST['Longitude'], dfTEST['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# FOUR SQUARE TIME

In [19]:
CLIENT_ID = 'IKXHMLECWUITBM1FBU2V4LGNQC1ZLHRA4N201COAUVYW1GV0' 
CLIENT_SECRET = 'I1T5UO4XT43HABW0QMAYBFTCYMD1IWHZ1TLCQ4NOW0EXZCYF' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print(VERSION)

Your credentails:
CLIENT_ID: IKXHMLECWUITBM1FBU2V4LGNQC1ZLHRA4N201COAUVYW1GV0
CLIENT_SECRET:I1T5UO4XT43HABW0QMAYBFTCYMD1IWHZ1TLCQ4NOW0EXZCYF
20180605


In [20]:
neighborhood_latitude = dfTEST.loc[0, 'Latitude'] 
neighborhood_longitude = dfTEST.loc[0, 'Longitude']

neighborhood_name = dfTEST.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [21]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 750 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=IKXHMLECWUITBM1FBU2V4LGNQC1ZLHRA4N201COAUVYW1GV0&client_secret=I1T5UO4XT43HABW0QMAYBFTCYMD1IWHZ1TLCQ4NOW0EXZCYF&v=20180605&ll=43.7532586,-79.3296565&radius=750&limit=100'

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f4c4fa23649c121d4ce3d66'},
 'response': {'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.760008606750006,
    'lng': -79.32032910977391},
   'sw': {'lat': 43.74650859325, 'lng': -79.33898389022609}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'distance': 245,
        'cc': 'CA

In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,DVP at York Mills,Road,43.758899,-79.334099
3,TTC Stop #09083,Bus Stop,43.759655,-79.332223


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return important information for venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=dfTEST['Neighborhood'],
                                   latitudes=dfTEST['Latitude'],
                                   longitudes=dfTEST['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(3656, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,DVP at York Mills,43.758899,-79.334099,Road
3,Parkwoods,43.753259,-79.329656,TTC Stop #09083,43.759655,-79.332223,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [28]:
#manhattan_venues.groupby('Neighborhood').count()

In [39]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
#print(toronto_onehot.columns[189])
# fixes the columns
fixed_columns = [toronto_onehot.columns[211]] + list(toronto_onehot.columns[:211])  + list(toronto_onehot.columns[212:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()



,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermar

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                      venue  freq
0           Badminton Court  0.09
1            Discount Store  0.09
2               Supermarket  0.09
3          Sushi Restaurant  0.09
4  Mediterranean Restaurant  0.09


----Alderwood, Long Branch----
               venue  freq
0        Pizza Place  0.15
1  Convenience Store  0.15
2               Park  0.08
3         Donut Shop  0.08
4        Gas Station  0.08


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0               Bank  0.09
1        Pizza Place  0.09
2        Coffee Shop  0.09
3  Mobile Phone Shop  0.05
4           Pharmacy  0.05


----Bayview Village----
                 venue  freq
0                 Bank   0.2
1  Japanese Restaurant   0.2
2        Shopping Mall   0.1
3              Dog Run   0.1
4                 Café   0.1


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2      Sandwich Place  0.06
3      

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(60)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Agincourt,Seafood Restaurant,Breakfast Spot,Discount Store,Latin American Restaurant,Sushi Restaurant,Lounge,Supermarket
1,"Alderwood, Long Branch",Pizza Place,Convenience Store,Pub,Donut Shop,Gas Station,Sandwich Place,Park
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Bank,Coffee Shop,Fried Chicken Joint,Restaurant,Shopping Mall,Sandwich Place
3,Bayview Village,Bank,Japanese Restaurant,Shopping Mall,Intersection,Dog Run,Grocery Store,Chinese Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Hobby Shop,Sushi Restaurant,Restaurant,Bagel Shop
5,Berczy Park,Coffee Shop,Café,Restaurant,Beer Bar,Hotel,Pub,Japanese Restaurant
6,"Birch Cliff, Cliffside West",Park,Diner,College Stadium,Café,Thai Restaurant,Skating Rink,General Entertainment
7,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Restaurant,Gift Shop,Bar,Bakery,Nightclub
8,"Business reply mail Processing Centre, South C...",Fast Food Restaurant,Restaurant,Italian Restaurant,Park,Light Rail Station,Brewery,Bar
9,"CN Tower, King and Spadina, Railway Lands, Har...",Harbor / Marina,Rental Car Location,Airport Terminal,Airport Service,Airport Lounge,Coffee Shop,Boat or Ferry


# Making KNN

In [107]:
# set number of clusters
kclusters = 7
#neighborhoods_venues_sorted
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=3).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([0, 6, 6, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 6, 4, 0, 0, 6, 0,
       0, 0, 6, 0, 0, 6, 0, 0, 1, 0, 6, 0, 4, 0, 0, 6, 0, 0, 0, 0, 6, 0,
       0, 6, 0, 1, 0, 0], dtype=int32)

In [108]:
#neighborhoods_venues_sorted.drop(columns=['Cluster Labels'],axis=1,inplace=True)


neighborhoods_venues_sorted.head(6)
toronto_merged = dfTEST
toronto_merged.head(6)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242


In [109]:
# add clustering labels
try:
    neighborhoods_venues_sorted.drop(columns=['Cluster Labels'],axis=1,inplace=True)
except:
    print('hi')
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfTEST
#toronto_merged.drop(5,inplace = True)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(14) 
#df.sort_values(by=['col1'])
toronto_merged.head(70).sort_values(by=['Cluster Labels'])

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
34,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,0.0,Massage Studio,Road,Fast Food Restaurant,Frame Store,Bar,Bank,Automotive Shop
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0.0,Coffee Shop,Café,Hotel,Clothing Store,Theater,Bar,Restaurant
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0.0,Coffee Shop,Portuguese Restaurant,Pharmacy,Bar,Park,Bakery,Gym
33,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0.0,Coffee Shop,Clothing Store,Japanese Restaurant,Juice Bar,Bank,Restaurant,Pharmacy
35,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,0.0,Coffee Shop,Café,Pizza Place,Bakery,Beer Bar,Fast Food Restaurant,Gastropub
36,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,0.0,Coffee Shop,Hotel,Restaurant,Café,Aquarium,Boat or Ferry,Brewery
37,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,0.0,Bar,Restaurant,Café,Coffee Shop,Asian Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant
38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,0.0,Coffee Shop,Bank,Rental Car Location,Department Store,Intersection,Discount Store,Pharmacy
39,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Bank,Japanese Restaurant,Shopping Mall,Intersection,Dog Run,Grocery Store,Chinese Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0.0,Greek Restaurant,Coffee Shop,Pub,Café,Fast Food Restaurant,Italian Restaurant,Furniture / Home Store


In [110]:
#toronto_merged.head(50).sort_values(by=['Cluster Labels'])

In [111]:
color=4

map_clusters = folium.Map(location=[torLat, torLon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    try:
        color=int(cluster)
    except:
        color=4
    #print(color)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[color-1],
        fill=True,
        fill_color=rainbow[color-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Analysis

From looking at each clustering labels, you can see their is a generally grouping of venue category that makes each label. Some examples are label 0 with most seen venues as Coffee shops,Cafes and Indian resturants. Label 1 with venues like Parks, Bus lines,and Bus stops. Label 4 with Pizza place and Resturants.Label 2,3 and 6 didnt have a lot of points cluster because they are more different than alike than other neighborhoods. Lastly label 6 had most veneues with Pizza place, Italin resturants, and Coffee shops.